# Unit 6: Outlier Detection and Treatment - Practical Exercises

## Table of Contents
1. [Introduction](#introduction)
2. [Setup](#setup)
3. [Visual Outlier Detection](#visual-detection)
4. [IQR Method](#iqr-method)
5. [Z-Score Method](#zscore-method)
6. [Modified Z-Score Method](#modified-zscore)
7. [Advanced Methods](#advanced-methods)
8. [Multivariate Outlier Detection](#multivariate-detection)
9. [Outlier Treatment Strategies](#treatment-strategies)
10. [Complete Workflow](#complete-workflow)

---

## 1. Introduction <a id='introduction'></a>

In this practical notebook, you'll learn to detect and treat outliers using various methods. You'll understand when to use each method and how to make informed decisions about outlier treatment.

**Learning Objectives:**
- Visualize outliers using box plots, scatter plots, and histograms
- Apply statistical methods (IQR, Z-score, Modified Z-score)
- Use advanced machine learning methods (Isolation Forest, LOF)
- Detect multivariate outliers
- Implement appropriate treatment strategies
- Build complete outlier detection workflows

## 2. Setup <a id='setup'></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import zscore, chi2
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler

# Settings
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
np.random.seed(42)

print("✅ Setup complete!")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

In [ ]:
# Load datasets
tips = sns.load_dataset('tips')
diamonds = sns.load_dataset('diamonds')

print("DATASETS LOADED")
print("="*80)
print(f"Tips dataset: {tips.shape}")
print(f"Diamonds dataset: {diamonds.shape}")

print("\nTips dataset sample:")
print(tips.head())
print("\nTips statistics:")
print(tips.describe())

## 3. Visual Outlier Detection <a id='visual-detection'></a>

Let's start by visualizing outliers.

In [ ]:
# Box plot for outlier detection
print("VISUAL OUTLIER DETECTION: Total Bill")
print("="*80)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Box plot
bp = axes[0].boxplot(tips['total_bill'], vert=True, patch_artist=True,
                      boxprops=dict(facecolor='lightblue', alpha=0.7))
axes[0].set_ylabel('Total Bill ($)', fontsize=11)
axes[0].set_title('Box Plot: Total Bill', fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Annotate box plot components
Q1 = tips['total_bill'].quantile(0.25)
Q2 = tips['total_bill'].quantile(0.50)
Q3 = tips['total_bill'].quantile(0.75)
IQR = Q3 - Q1

axes[0].text(1.3, Q1, f'Q1: ${Q1:.2f}', verticalalignment='center', fontsize=9)
axes[0].text(1.3, Q2, f'Q2: ${Q2:.2f}', verticalalignment='center', fontsize=9)
axes[0].text(1.3, Q3, f'Q3: ${Q3:.2f}', verticalalignment='center', fontsize=9)

# Histogram
axes[1].hist(tips['total_bill'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[1].axvline(tips['total_bill'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: ${tips["total_bill"].mean():.2f}')
axes[1].axvline(tips['total_bill'].median(), color='green', linestyle='--', 
                linewidth=2, label=f'Median: ${tips["total_bill"].median():.2f}')
axes[1].set_xlabel('Total Bill ($)', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].set_title('Histogram: Total Bill', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

# Scatter plot (index vs value)
axes[2].scatter(range(len(tips)), tips['total_bill'], alpha=0.6)
axes[2].axhline(tips['total_bill'].mean(), color='red', linestyle='--', alpha=0.5, label='Mean')
axes[2].axhline(tips['total_bill'].mean() + 3*tips['total_bill'].std(), 
                color='orange', linestyle=':', alpha=0.5, label='Mean ± 3σ')
axes[2].axhline(tips['total_bill'].mean() - 3*tips['total_bill'].std(), 
                color='orange', linestyle=':', alpha=0.5)
axes[2].set_xlabel('Index', fontsize=11)
axes[2].set_ylabel('Total Bill ($)', fontsize=11)
axes[2].set_title('Scatter Plot: Total Bill', fontsize=12, fontweight='bold')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nBox Plot Components:")
print(f"  Q1 (25th percentile): ${Q1:.2f}")
print(f"  Q2 (50th percentile): ${Q2:.2f}")
print(f"  Q3 (75th percentile): ${Q3:.2f}")
print(f"  IQR (Q3 - Q1): ${IQR:.2f}")
print(f"  Lower fence (Q1 - 1.5×IQR): ${Q1 - 1.5*IQR:.2f}")
print(f"  Upper fence (Q3 + 1.5×IQR): ${Q3 + 1.5*IQR:.2f}")

## 4. IQR Method <a id='iqr-method'></a>

The most common statistical method for outlier detection.

In [ ]:
def detect_outliers_iqr(data, k=1.5):
    """
    Detect outliers using IQR method.
    
    Parameters:
    -----------
    data : pd.Series
        Data to check for outliers
    k : float, default=1.5
        Multiplier for IQR (1.5 for mild, 3.0 for extreme)
        
    Returns:
    --------
    outliers : boolean array
        True for outliers, False otherwise
    lower_bound : float
        Lower threshold
    upper_bound : float
        Upper threshold
    """
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - k * IQR
    upper_bound = Q3 + k * IQR
    
    outliers = (data < lower_bound) | (data > upper_bound)
    
    return outliers, lower_bound, upper_bound


# Apply IQR method to total_bill
print("IQR METHOD: Total Bill")
print("="*80)

outliers, lower, upper = detect_outliers_iqr(tips['total_bill'], k=1.5)

print(f"\nThresholds:")
print(f"  Lower bound: ${lower:.2f}")
print(f"  Upper bound: ${upper:.2f}")

print(f"\nOutlier Detection:")
print(f"  Number of outliers: {outliers.sum()}")
print(f"  Percentage: {outliers.mean()*100:.2f}%")

# Show outlier values
print(f"\nOutlier Values:")
outlier_values = tips.loc[outliers, 'total_bill'].sort_values(ascending=False)
print(outlier_values.head(10))

# Statistics comparison
print(f"\nWith vs Without Outliers:")
print(f"  Mean (all data): ${tips['total_bill'].mean():.2f}")
print(f"  Mean (no outliers): ${tips.loc[~outliers, 'total_bill'].mean():.2f}")
print(f"  Median (all data): ${tips['total_bill'].median():.2f}")
print(f"  Median (no outliers): ${tips.loc[~outliers, 'total_bill'].median():.2f}")

In [ ]:
# Visualize IQR outlier detection
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram with bounds
axes[0].hist(tips['total_bill'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(lower, color='red', linestyle='--', linewidth=2, label=f'Lower: ${lower:.2f}')
axes[0].axvline(upper, color='red', linestyle='--', linewidth=2, label=f'Upper: ${upper:.2f}')
axes[0].set_xlabel('Total Bill ($)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('IQR Method: Outlier Bounds', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Scatter plot with outliers highlighted
colors = ['red' if x else 'blue' for x in outliers]
axes[1].scatter(range(len(tips)), tips['total_bill'], c=colors, alpha=0.6)
axes[1].axhline(lower, color='red', linestyle='--', alpha=0.5)
axes[1].axhline(upper, color='red', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Index', fontsize=11)
axes[1].set_ylabel('Total Bill ($)', fontsize=11)
axes[1].set_title('Outliers Highlighted (Red)', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Compare different k values
print("\nCOMPARING DIFFERENT K VALUES")
print("="*80)

k_values = [1.5, 2.0, 3.0]

for k in k_values:
    outliers, lower, upper = detect_outliers_iqr(tips['total_bill'], k=k)
    print(f"\nk = {k}:")
    print(f"  Bounds: [${lower:.2f}, ${upper:.2f}]")
    print(f"  Outliers: {outliers.sum()} ({outliers.mean()*100:.2f}%)")

## 5. Z-Score Method <a id='zscore-method'></a>

Suitable for normally distributed data.

In [ ]:
def detect_outliers_zscore(data, threshold=3):
    """
    Detect outliers using Z-score method.
    
    Parameters:
    -----------
    data : pd.Series
        Data to check for outliers
    threshold : float, default=3
        Z-score threshold (typically 2.5 or 3)
        
    Returns:
    --------
    outliers : boolean array
        True for outliers, False otherwise
    z_scores : array
        Z-scores for each data point
    """
    z_scores = np.abs(zscore(data.dropna()))
    outliers = z_scores > threshold
    
    return outliers, z_scores


# Apply Z-score method
print("Z-SCORE METHOD: Total Bill")
print("="*80)

outliers_z, z_scores = detect_outliers_zscore(tips['total_bill'], threshold=3)

print(f"\nThreshold: Z > 3")
print(f"Number of outliers: {outliers_z.sum()}")
print(f"Percentage: {outliers_z.mean()*100:.2f}%")

# Show outliers
print(f"\nOutliers (Z-score > 3):")
outlier_df = tips[outliers_z].copy()
outlier_df['z_score'] = z_scores[outliers_z]
print(outlier_df[['total_bill', 'z_score']].sort_values('z_score', ascending=False))

# Statistics
print(f"\nZ-Score Statistics:")
print(f"  Mean Z-score: {z_scores.mean():.3f}")
print(f"  Std Z-score: {z_scores.std():.3f}")
print(f"  Max Z-score: {z_scores.max():.3f}")

In [ ]:
# Visualize Z-score distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Z-score distribution
axes[0].hist(z_scores, bins=30, edgecolor='black', alpha=0.7, color='coral')
axes[0].axvline(3, color='red', linestyle='--', linewidth=2, label='Threshold (Z=3)')
axes[0].set_xlabel('Z-Score', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Distribution of Z-Scores', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Original data with Z-score coloring
scatter = axes[1].scatter(range(len(tips)), tips['total_bill'], 
                          c=z_scores, cmap='RdYlGn_r', alpha=0.6, s=50)
axes[1].set_xlabel('Index', fontsize=11)
axes[1].set_ylabel('Total Bill ($)', fontsize=11)
axes[1].set_title('Data Points Colored by Z-Score', fontsize=12, fontweight='bold')
cbar = plt.colorbar(scatter, ax=axes[1])
cbar.set_label('Z-Score', fontsize=10)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Compare IQR vs Z-score methods
print("\nCOMPARING IQR vs Z-SCORE")
print("="*80)

outliers_iqr, _, _ = detect_outliers_iqr(tips['total_bill'], k=1.5)
outliers_z, _ = detect_outliers_zscore(tips['total_bill'], threshold=3)

print(f"IQR method (k=1.5): {outliers_iqr.sum()} outliers ({outliers_iqr.mean()*100:.2f}%)")
print(f"Z-score method (threshold=3): {outliers_z.sum()} outliers ({outliers_z.mean()*100:.2f}%)")

# Overlap
both = outliers_iqr & outliers_z
either = outliers_iqr | outliers_z
only_iqr = outliers_iqr & ~outliers_z
only_z = outliers_z & ~outliers_iqr

print(f"\nOverlap Analysis:")
print(f"  Detected by both: {both.sum()}")
print(f"  Detected by either: {either.sum()}")
print(f"  Only IQR: {only_iqr.sum()}")
print(f"  Only Z-score: {only_z.sum()}")

## 6. Modified Z-Score Method <a id='modified-zscore'></a>

Robust version using median and MAD.

In [ ]:
def detect_outliers_modified_zscore(data, threshold=3.5):
    """
    Detect outliers using Modified Z-score method (robust).
    
    Parameters:
    -----------
    data : pd.Series
        Data to check for outliers
    threshold : float, default=3.5
        Modified Z-score threshold
        
    Returns:
    --------
    outliers : boolean array
        True for outliers, False otherwise
    modified_z_scores : array
        Modified Z-scores
    """
    median = np.median(data)
    mad = np.median(np.abs(data - median))
    
    # Avoid division by zero
    if mad == 0:
        mad = np.mean(np.abs(data - median))
    
    modified_z_scores = 0.6745 * (data - median) / mad
    outliers = np.abs(modified_z_scores) > threshold
    
    return outliers, modified_z_scores


# Apply Modified Z-score method
print("MODIFIED Z-SCORE METHOD: Total Bill")
print("="*80)

outliers_mz, mod_z_scores = detect_outliers_modified_zscore(tips['total_bill'], threshold=3.5)

print(f"\nThreshold: Modified Z > 3.5")
print(f"Number of outliers: {outliers_mz.sum()}")
print(f"Percentage: {outliers_mz.mean()*100:.2f}%")

# Compare with standard Z-score
outliers_z, z_scores = detect_outliers_zscore(tips['total_bill'], threshold=3)

print(f"\nComparison:")
print(f"  Standard Z-score: {outliers_z.sum()} outliers")
print(f"  Modified Z-score: {outliers_mz.sum()} outliers")

# Show differences
print(f"\nWhy use Modified Z-score?")
print(f"  Standard uses: Mean & Std Dev (sensitive to outliers)")
print(f"  Modified uses: Median & MAD (robust to outliers)")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Standard Z-score
axes[0].hist(z_scores, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(3, color='red', linestyle='--', linewidth=2, label='Threshold')
axes[0].set_xlabel('Standard Z-Score', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Standard Z-Score Distribution', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Modified Z-score
axes[1].hist(mod_z_scores, bins=30, edgecolor='black', alpha=0.7, color='coral')
axes[1].axvline(3.5, color='red', linestyle='--', linewidth=2, label='Threshold')
axes[1].axvline(-3.5, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Modified Z-Score', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].set_title('Modified Z-Score Distribution', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Advanced Methods <a id='advanced-methods'></a>

Machine learning approaches for outlier detection.

In [ ]:
# Isolation Forest
print("ISOLATION FOREST METHOD")
print("="*80)

# Prepare data
X = tips[['total_bill']].values

# Fit Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42)
predictions = iso_forest.fit_predict(X)

# -1 for outliers, 1 for inliers
outliers_if = predictions == -1

print(f"\nContamination: 5% (expected outlier rate)")
print(f"Outliers detected: {outliers_if.sum()} ({outliers_if.mean()*100:.2f}%)")

# Get anomaly scores
scores = iso_forest.score_samples(X)
print(f"\nAnomaly Scores:")
print(f"  Mean: {scores.mean():.3f}")
print(f"  Min (most anomalous): {scores.min():.3f}")
print(f"  Max (most normal): {scores.max():.3f}")

# Show detected outliers
print(f"\nDetected Outliers:")
outlier_data = tips[outliers_if][['total_bill', 'tip', 'size']]
outlier_data['anomaly_score'] = scores[outliers_if]
print(outlier_data.sort_values('anomaly_score').head(10))

In [ ]:
# Visualize Isolation Forest results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram with outliers highlighted
axes[0].hist(tips.loc[~outliers_if, 'total_bill'], bins=30, 
             edgecolor='black', alpha=0.7, color='steelblue', label='Normal')
axes[0].hist(tips.loc[outliers_if, 'total_bill'], bins=10, 
             edgecolor='black', alpha=0.7, color='red', label='Outliers')
axes[0].set_xlabel('Total Bill ($)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Isolation Forest: Detected Outliers', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Anomaly scores
scatter = axes[1].scatter(range(len(tips)), tips['total_bill'], 
                          c=scores, cmap='RdYlGn', alpha=0.6, s=50)
axes[1].set_xlabel('Index', fontsize=11)
axes[1].set_ylabel('Total Bill ($)', fontsize=11)
axes[1].set_title('Anomaly Scores (Red = More Anomalous)', fontsize=12, fontweight='bold')
cbar = plt.colorbar(scatter, ax=axes[1])
cbar.set_label('Anomaly Score', fontsize=10)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Local Outlier Factor (LOF)
print("\nLOCAL OUTLIER FACTOR (LOF)")
print("="*80)

# Fit LOF
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
predictions_lof = lof.fit_predict(X)

outliers_lof = predictions_lof == -1
lof_scores = lof.negative_outlier_factor_

print(f"\nParameters:")
print(f"  n_neighbors: 20")
print(f"  contamination: 5%")

print(f"\nOutliers detected: {outliers_lof.sum()} ({outliers_lof.mean()*100:.2f}%)")

print(f"\nLOF Scores:")
print(f"  Mean: {lof_scores.mean():.3f}")
print(f"  Min (most outlier-like): {lof_scores.min():.3f}")
print(f"  Max: {lof_scores.max():.3f}")

# Show outliers
print(f"\nDetected Outliers:")
outlier_data_lof = tips[outliers_lof][['total_bill', 'tip']]
outlier_data_lof['lof_score'] = lof_scores[outliers_lof]
print(outlier_data_lof.sort_values('lof_score').head(10))

## 8. Multivariate Outlier Detection <a id='multivariate-detection'></a>

Detecting outliers across multiple variables.

In [ ]:
# Multivariate outlier detection using Isolation Forest
print("MULTIVARIATE OUTLIER DETECTION")
print("="*80)

# Select multiple features
features = ['total_bill', 'tip']
X_multi = tips[features].values

# Fit Isolation Forest
iso_forest_multi = IsolationForest(contamination=0.05, random_state=42)
predictions_multi = iso_forest_multi.fit_predict(X_multi)
outliers_multi = predictions_multi == -1

print(f"\nFeatures used: {features}")
print(f"Outliers detected: {outliers_multi.sum()} ({outliers_multi.mean()*100:.2f}%)")

# Show multivariate outliers
print(f"\nMultivariate Outliers:")
print(tips[outliers_multi][features].head(10))

# Compare with univariate detection
outliers_univar, _, _ = detect_outliers_iqr(tips['total_bill'], k=1.5)

print(f"\nComparison:")
print(f"  Univariate (total_bill only): {outliers_univar.sum()} outliers")
print(f"  Multivariate (total_bill + tip): {outliers_multi.sum()} outliers")
print(f"  Overlap: {(outliers_univar & outliers_multi).sum()}")
print(f"  Only multivariate: {(outliers_multi & ~outliers_univar).sum()}")

In [ ]:
# Visualize multivariate outliers
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Scatter plot - all data
axes[0].scatter(tips.loc[~outliers_multi, 'total_bill'], 
                tips.loc[~outliers_multi, 'tip'],
                alpha=0.6, color='blue', label='Normal', s=50)
axes[0].scatter(tips.loc[outliers_multi, 'total_bill'], 
                tips.loc[outliers_multi, 'tip'],
                alpha=0.8, color='red', label='Outliers', s=100, marker='X')
axes[0].set_xlabel('Total Bill ($)', fontsize=11)
axes[0].set_ylabel('Tip ($)', fontsize=11)
axes[0].set_title('Multivariate Outliers (Isolation Forest)', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# 3D scatter if we had 3 features - here show size relationship
axes[1].scatter(tips.loc[~outliers_multi, 'total_bill'], 
                tips.loc[~outliers_multi, 'tip'],
                s=tips.loc[~outliers_multi, 'size']*50,
                alpha=0.6, color='blue', label='Normal')
axes[1].scatter(tips.loc[outliers_multi, 'total_bill'], 
                tips.loc[outliers_multi, 'tip'],
                s=tips.loc[outliers_multi, 'size']*50,
                alpha=0.8, color='red', label='Outliers', marker='X')
axes[1].set_xlabel('Total Bill ($)', fontsize=11)
axes[1].set_ylabel('Tip ($)', fontsize=11)
axes[1].set_title('Size-Weighted View (Size = Bubble Size)', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Outlier Treatment Strategies <a id='treatment-strategies'></a>

Different ways to handle detected outliers.

In [ ]:
# Strategy 1: Deletion
print("STRATEGY 1: DELETION")
print("="*80)

# Detect outliers
outliers, lower, upper = detect_outliers_iqr(tips['total_bill'], k=1.5)

# Remove outliers
tips_no_outliers = tips[~outliers].copy()

print(f"\nOriginal dataset: {len(tips)} rows")
print(f"After deletion: {len(tips_no_outliers)} rows")
print(f"Removed: {outliers.sum()} rows ({outliers.mean()*100:.2f}%)")

print(f"\nStatistics Before & After:")
print(f"  Mean: ${tips['total_bill'].mean():.2f} → ${tips_no_outliers['total_bill'].mean():.2f}")
print(f"  Std:  ${tips['total_bill'].std():.2f} → ${tips_no_outliers['total_bill'].std():.2f}")
print(f"  Median: ${tips['total_bill'].median():.2f} → ${tips_no_outliers['total_bill'].median():.2f}")

In [ ]:
# Strategy 2: Capping/Winsorization
print("\nSTRATEGY 2: CAPPING/WINSORIZATION")
print("="*80)

def cap_outliers(data, column, lower_percentile=5, upper_percentile=95):
    """
    Cap outliers at specified percentiles.
    """
    lower_cap = data[column].quantile(lower_percentile / 100)
    upper_cap = data[column].quantile(upper_percentile / 100)
    
    data_capped = data.copy()
    data_capped[column + '_capped'] = data_capped[column].clip(lower=lower_cap, upper=upper_cap)
    
    return data_capped, lower_cap, upper_cap


# Apply capping
tips_capped, lower_cap, upper_cap = cap_outliers(tips, 'total_bill', 
                                                   lower_percentile=5, upper_percentile=95)

print(f"\nCapping at 5th and 95th percentiles:")
print(f"  Lower cap: ${lower_cap:.2f}")
print(f"  Upper cap: ${upper_cap:.2f}")

values_capped = (tips['total_bill'] != tips_capped['total_bill_capped']).sum()
print(f"  Values capped: {values_capped} ({values_capped/len(tips)*100:.2f}%)")

print(f"\nStatistics Before & After:")
print(f"  Mean: ${tips['total_bill'].mean():.2f} → ${tips_capped['total_bill_capped'].mean():.2f}")
print(f"  Max:  ${tips['total_bill'].max():.2f} → ${tips_capped['total_bill_capped'].max():.2f}")
print(f"  Min:  ${tips['total_bill'].min():.2f} → ${tips_capped['total_bill_capped'].min():.2f}")

In [ ]:
# Strategy 3: Transformation
print("\nSTRATEGY 3: LOG TRANSFORMATION")
print("="*80)

# Apply log transformation
tips['total_bill_log'] = np.log1p(tips['total_bill'])

# Check for outliers after transformation
outliers_original, _, _ = detect_outliers_iqr(tips['total_bill'], k=1.5)
outliers_log, _, _ = detect_outliers_iqr(tips['total_bill_log'], k=1.5)

print(f"\nOutliers before transformation: {outliers_original.sum()}")
print(f"Outliers after log transformation: {outliers_log.sum()}")
print(f"Reduction: {outliers_original.sum() - outliers_log.sum()} outliers")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Before transformation
axes[0].hist(tips['total_bill'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_xlabel('Total Bill ($)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Before Transformation', fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# After transformation
axes[1].hist(tips['total_bill_log'], bins=30, edgecolor='black', alpha=0.7, color='coral')
axes[1].set_xlabel('Log(Total Bill)', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].set_title('After Log Transformation', fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Compare all strategies
print("\nCOMPARING ALL TREATMENT STRATEGIES")
print("="*80)

strategies = {
    'Original': tips['total_bill'],
    'Deletion': tips_no_outliers['total_bill'],
    'Capped': tips_capped['total_bill_capped'],
    'Log Transform': np.expm1(tips['total_bill_log'])  # Back-transform for comparison
}

print(f"\n{'Strategy':<20} {'Mean':>10} {'Std':>10} {'Median':>10} {'N':>6}")
print("-" * 60)
for name, data in strategies.items():
    print(f"{name:<20} ${data.mean():>9.2f} ${data.std():>9.2f} ${data.median():>9.2f} {len(data):>6}")

## 10. Complete Workflow <a id='complete-workflow'></a>

Building a comprehensive outlier detection and treatment pipeline.

In [ ]:
def outlier_analysis_pipeline(data, column, treatment='report'):
    """
    Complete outlier detection and treatment pipeline.
    
    Parameters:
    -----------
    data : DataFrame
        Input data
    column : str
        Column to analyze
    treatment : str
        Treatment method: 'report', 'delete', 'cap', 'transform'
        
    Returns:
    --------
    results : dict
        Analysis results and treated data
    """
    results = {}
    
    print("="*80)
    print(f"OUTLIER ANALYSIS PIPELINE: {column}")
    print("="*80)
    
    # 1. Visual inspection
    print("\n1. VISUAL INSPECTION")
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # Box plot
    bp = axes[0].boxplot(data[column].dropna(), vert=True, patch_artist=True,
                          boxprops=dict(facecolor='lightblue'))
    axes[0].set_ylabel(column, fontsize=11)
    axes[0].set_title('Box Plot', fontsize=12, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Histogram
    axes[1].hist(data[column].dropna(), bins=30, edgecolor='black', alpha=0.7)
    axes[1].set_xlabel(column, fontsize=11)
    axes[1].set_ylabel('Frequency', fontsize=11)
    axes[1].set_title('Histogram', fontsize=12, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    # Scatter
    axes[2].scatter(range(len(data)), data[column], alpha=0.6)
    axes[2].set_xlabel('Index', fontsize=11)
    axes[2].set_ylabel(column, fontsize=11)
    axes[2].set_title('Scatter Plot', fontsize=12, fontweight='bold')
    axes[2].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 2. Statistical detection
    print("\n2. STATISTICAL DETECTION")
    print("-"*80)
    
    # IQR method
    outliers_iqr, lower_iqr, upper_iqr = detect_outliers_iqr(data[column], k=1.5)
    print(f"IQR Method (k=1.5):")
    print(f"  Bounds: [{lower_iqr:.2f}, {upper_iqr:.2f}]")
    print(f"  Outliers: {outliers_iqr.sum()} ({outliers_iqr.mean()*100:.2f}%)")
    
    # Z-score method
    outliers_z, z_scores = detect_outliers_zscore(data[column], threshold=3)
    print(f"\nZ-Score Method (threshold=3):")
    print(f"  Outliers: {outliers_z.sum()} ({outliers_z.mean()*100:.2f}%)")
    
    # Modified Z-score
    outliers_mz, _ = detect_outliers_modified_zscore(data[column], threshold=3.5)
    print(f"\nModified Z-Score Method (threshold=3.5):")
    print(f"  Outliers: {outliers_mz.sum()} ({outliers_mz.mean()*100:.2f}%)")
    
    # Isolation Forest
    X = data[[column]].values
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    outliers_if = iso_forest.fit_predict(X) == -1
    print(f"\nIsolation Forest (contamination=0.05):")
    print(f"  Outliers: {outliers_if.sum()} ({outliers_if.mean()*100:.2f}%)")
    
    # Consensus outliers (detected by multiple methods)
    consensus = outliers_iqr & outliers_z
    print(f"\nConsensus (IQR + Z-score):")
    print(f"  Outliers: {consensus.sum()} ({consensus.mean()*100:.2f}%)")
    
    results['outliers'] = consensus
    
    # 3. Impact assessment
    print("\n3. IMPACT ASSESSMENT")
    print("-"*80)
    
    mean_with = data[column].mean()
    mean_without = data.loc[~consensus, column].mean()
    median_with = data[column].median()
    median_without = data.loc[~consensus, column].median()
    
    print(f"Mean: {mean_with:.2f} → {mean_without:.2f} (change: {mean_without-mean_with:.2f})")
    print(f"Median: {median_with:.2f} → {median_without:.2f} (change: {median_without-median_with:.2f})")
    
    results['impact'] = {
        'mean_change': mean_without - mean_with,
        'median_change': median_without - median_with
    }
    
    # 4. Treatment
    print(f"\n4. TREATMENT: {treatment.upper()}")
    print("-"*80)
    
    if treatment == 'delete':
        treated_data = data[~consensus].copy()
        print(f"Removed {consensus.sum()} outliers")
        results['data'] = treated_data
    
    elif treatment == 'cap':
        lower_cap = data[column].quantile(0.05)
        upper_cap = data[column].quantile(0.95)
        treated_data = data.copy()
        treated_data[column + '_treated'] = treated_data[column].clip(lower=lower_cap, upper=upper_cap)
        print(f"Capped at [{lower_cap:.2f}, {upper_cap:.2f}]")
        results['data'] = treated_data
    
    elif treatment == 'transform':
        treated_data = data.copy()
        treated_data[column + '_treated'] = np.log1p(treated_data[column])
        print(f"Applied log transformation")
        results['data'] = treated_data
    
    else:  # report
        print(f"Reporting only - no treatment applied")
        results['data'] = data
    
    print("\n" + "="*80)
    print("✅ ANALYSIS COMPLETE")
    print("="*80)
    
    return results


# Run the pipeline
results = outlier_analysis_pipeline(tips, 'total_bill', treatment='report')

In [ ]:
# Apply to multiple columns
print("\nANALYZING MULTIPLE COLUMNS")
print("="*80)

columns_to_analyze = ['total_bill', 'tip']

for col in columns_to_analyze:
    print(f"\n{'='*80}")
    print(f"COLUMN: {col}")
    print('='*80)
    
    # Quick analysis
    outliers_iqr, lower, upper = detect_outliers_iqr(tips[col], k=1.5)
    outliers_z, _ = detect_outliers_zscore(tips[col], threshold=3)
    
    print(f"IQR outliers: {outliers_iqr.sum()} ({outliers_iqr.mean()*100:.2f}%)")
    print(f"Z-score outliers: {outliers_z.sum()} ({outliers_z.mean()*100:.2f}%)")
    print(f"Bounds: [{lower:.2f}, {upper:.2f}]")

## Summary

**What We Accomplished:**

1. ✅ Visualized outliers using box plots, histograms, and scatter plots
2. ✅ Applied IQR method for robust outlier detection
3. ✅ Used Z-score method for normally distributed data
4. ✅ Applied Modified Z-score for robust detection
5. ✅ Implemented advanced methods (Isolation Forest, LOF)
6. ✅ Detected multivariate outliers
7. ✅ Compared different treatment strategies
8. ✅ Built complete outlier analysis pipeline

**Key Takeaways:**

- 🎯 **Always visualize first** - See patterns before applying statistics
- 🎯 **Use multiple methods** - Different methods for different scenarios
- 🎯 **Understand the cause** - Treatment depends on why outliers exist
- 🎯 **Assess impact** - Check how outliers affect your results
- 🎯 **Document decisions** - Record what you did and why
- 🎯 **Domain knowledge is critical** - Context determines treatment

**Method Selection Guide:**

| Data Type | Recommended Method |
|-----------|-------------------|
| Skewed data | IQR, Modified Z-score |
| Normal data | Z-score |
| Multivariate | Isolation Forest, LOF |
| Spatial data | DBSCAN |
| Unknown distribution | IQR + Isolation Forest |

**Next Steps:**

- Practice with different datasets
- Complete the exercises notebook
- Apply to your own data
- Build automated pipelines

**Congratulations!** You've mastered outlier detection and treatment! 🎉